<a href="https://colab.research.google.com/github/TandonArchit/LyricAI/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
pip install spleeter

In [ ]:
# Write file names
File_To_Convert = '[SHORT]LetMeDown.mp3'
File_Only_Name = '[SHORT]LetMeDown'
!spleeter separate -o output/ '/content/[SHORT]LetMeDown.mp3' 

INFO:spleeter:File output/[SHORT]LetMeDown/vocals.wav written succesfully
INFO:spleeter:File output/[SHORT]LetMeDown/accompaniment.wav written succesfully


In [ ]:
# Code to delete files
import os
# !rm -rf <folder_name>
# for i in range(1000):
#   var = str(str(i)+".mp4")
#   os.remove("/content/" + var)

In [ ]:
pip install moviepy

In [ ]:
pip install gizeh

In [ ]:
pip install bing_image_downloader

In [ ]:
pip install pydub

In [ ]:
# Splitting Audio File

from pydub import AudioSegment 
from pydub.utils import make_chunks 

myaudio = AudioSegment.from_file("/content/output/"+ File_Only_Name + "/vocals.wav", "wav") 
chunk_length_ms = 25000 
chunks = make_chunks(myaudio,chunk_length_ms)

for i, chunk in enumerate(chunks): 
    chunk_name = "{0}.wav".format(i) 
    print ("exporting", chunk_name) 
    chunk.export(chunk_name, format="wav") 

exporting 0.wav
exporting 1.wav


In [ ]:
# Transcripting Audio

import requests
import time


authKey = '17057cb9950542a6b379b6dc8613e09f'
headers = {
    'authorization' : authKey,
    'content-type'  : 'application/json'
}


uploadUrl      = 'https://api.assemblyai.com/v2/upload'
transcriptUrl  = 'https://api.assemblyai.com/v2/transcript'


def uploadMyFile(fileName):

    def _readMyFile(fn):

        chunkSize = 1000

        with open(fn, 'rb') as fileStream:
            while True:
                data = fileStream.read(chunkSize)
                if not data:
                    break
                yield data
    response = requests.post(
        uploadUrl,
        headers= headers,
        data= _readMyFile(fileName)
    )
    json = response.json()
    return json['upload_url']


def startTranscription(aurl):
    response = requests.post(
        transcriptUrl,
        headers= headers,
        json= { 'audio_url' : aurl }
    )   
    json = response.json()
    return json['id']


def getTranscription(tid):
    maxAttempts = 15
    timedout    = False
    while True:
        response = requests.get(
            f'{transcriptUrl}/{tid}',
            headers= headers
        )
        json = response.json()

        if json['status'] == 'completed':
            break
        maxAttempts -= 1
        timedout = maxAttempts <= 0  ## We are making sure that enough time is given for the API to respond and the function timesout incase of an infinite loop somewhere.

        if timedout:
            break
        time.sleep(3)
    return 'Timeout...' if timedout else json['text']

def final_assembly_AI():
  i = 0
  rtv = ''
  while True:
    try:
      audioUrl = uploadMyFile(str(i) + '.wav')
      transcriptionID = startTranscription(audioUrl)
      text = getTranscription(transcriptionID)
      rtv += text
    except:
      break
    i += 1 
  return rtv  

In [ ]:
# Image Resize

from PIL import Image

def change_size(location):

  im = Image.open(location)
  newsize = (720, 480) ## We are setting an image size because without it, if the code downloads a file that is too big for the video, it won't give a mapping error.
  im = im.resize(newsize)

In [ ]:
# Get Images From The Web

def get_image(x):

  from bing_image_downloader import downloader
  query_string = str(x)
## Our search paramenters are set here. The two main ones are: query_string and number of images to download, limit, is 1.  downloader.download(query_string, limit=1, output_dir='dataset', adult_filter_off=False, force_replace=False, timeout=60, verbose=True) 

In [ ]:
# Final Video Making

import moviepy.editor as mpy
import gizeh as gz
from moviepy.editor import *
import os
from natsort import natsorted
import time

def make_full_video(lyrics):

  def split_lyrics(lyrics):

    LyricsList = lyrics.split()
    n = 1
    final = [LyricsList[i * n:(i + 1) * n] for i in range((len(LyricsList) + n - 1) // n )] 
    return final

  count = (len(split_lyrics(lyrics)))

  def listToString(s): 
      first_string = " " 
      return (first_string.join(s))

  def make_video(lyrics,tempID):

    def render_text(t):

        Colour = (3/255, 160/255, 98/255)
        surface = gz.Surface(640, 60, bg_color=(24/255, 24/255, 24/255))
        text = gz.text(str(lyrics), fontfamily="Helvetica",fontsize=40, fontweight='bold', fill=Colour , xy=(320, 40))
        text.draw(surface)
        return surface.get_npimage()

    text = mpy.VideoClip(render_text, duration=0.6)

## The following if condition is put in place to make sure the video displays the word 'LyricAI' first and then start the song.    if tempID > 10: 

      get_image(lyrics)
      time.sleep(1)
      try:
        image_path = '/content/dataset/'+str(lyrics)+'/Image_1.jpg'
        change_size(image_path)
        image = mpy.ImageClip(image_path)
         
      except:
         try:
           try:
             image_path = '/content/dataset/'+str(lyrics)+'/Image_1.png'
             change_size(image_path)
             image = mpy.ImageClip(image_path)     
           except:  
             image_path = '/content/dataset/'+str(lyrics)+'/Image_1.jpeg'
             change_size(image_path)
             image = mpy.ImageClip(image_path)        
         except:
           a = 0
           while a == 0:
             time.sleep(1)
             try:
               image_path = '/content/dataset/'+str(lyrics)+'/Image_1.gif'
               change_size(image_path)
               image = mpy.ImageClip(image_path)
               a = 1
             except:
               pass
    else:
      get_image("Black Background")
      try:

        image_path = '/content/dataset/Black Background/Image_1.jpg'
        change_size(image_path)
        image = mpy.ImageClip(image_path)

      except:
        try:

          image_path = '/content/dataset/Black Background/Image_1.png'
          change_size(image_path)
          image = mpy.ImageClip(image_path)

        except:

          image_path = '/content/dataset/Black Background/Image_1.jpeg'
          change_size(image_path)
          image = mpy.ImageClip(image_path)  


           
## We have set the video resolution to 1920 x 1080, which means that the output will be officially Full HD.    VIDEO_SIZE = (1920, 1080)


    try:

      video = video = mpy.CompositeVideoClip(
          [
            image.set_position('center'),
              text.set_position(
                  ('center')
              )
          ],
          size=VIDEO_SIZE).\
          on_color(
              color=(24,24,24),
              col_opacity=1).set_duration(0.6)
      video.write_videofile(str(tempID)+ '.mp4', fps=10) 
    except:

## Here we form the layout for the video.      video = video = mpy.CompositeVideoClip(
          [
              text.set_position(
                  ('center')
              )
          ],
          size=VIDEO_SIZE).\
          on_color(
              color=(24,24,24),
              col_opacity=1).set_duration(0.5)
      video.write_videofile(str(tempID)+ '.mp4', fps=10) 

   
  ListOfLyrics= (split_lyrics(lyrics))  

  for i in range(count): 
    make_video(listToString(ListOfLyrics[i]),i) 

  Merging =[]

  for root, dirs, files in os.walk("/content"):
      files = natsorted(files)
      for file in files:
          if os.path.splitext(file)[1] == '.mp4':
              filePath = os.path.join(root, file)
              video = VideoFileClip(filePath)
              Merging.append(video)

  final_clip = concatenate_videoclips(Merging)
  final_clip.to_videofile("output.mp4", fps=24, remove_temp=False)

make_full_video('LyricAI LyricAI LyricAI LyricAI LyricAI LyricAI LyricAI LyricAI LyricAI LyricAI '+final_assembly_AI()) 

videoclip = VideoFileClip("output.mp4")

audioclip = AudioFileClip("/content/output/" + File_Only_Name + "/accompaniment.wav")

new_audioclip = CompositeAudioClip([audioclip])

videoclip.audio = new_audioclip

videoclip.write_videofile("Final.mp4")
# Final.mp4 is the final output

[%] Downloading Images to /content/dataset/Black Background


[!!]Indexing page: 1

[%] Indexed 1 Images on Page 1.


[%] Downloading Image #1 from https://cdn.wallpapersafari.com/54/57/VFmTa6.png
[%] File Downloaded !



[%] Done. Downloaded 1 images.

Please show your support here
https://www.buymeacoffee.com/gurugaurav


[MoviePy] >>>> Building video 0.mp4
[MoviePy] Writing video 0.mp4


 86%|████████▌ | 6/7 [00:02<00:00,  2.02it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: 0.mp4 

[%] Downloading Images to /content/dataset/Black Background


[!!]Indexing page: 1

[%] Indexed 1 Images on Page 1.


[%] Downloading Image #1 from https://cdn.wallpapersafari.com/54/57/VFmTa6.png
[%] File Downloaded !



[%] Done. Downloaded 1 images.

Please show your support here
https://www.buymeacoffee.com/gurugaurav


[MoviePy] >>>> Building video 1.mp4
[MoviePy] Writing video 1.mp4


 86%|████████▌ | 6/7 [00:02<00:00,  2.02it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: 1.mp4 

[%] Downloading Images to /content/dataset/Black Background


[!!]Indexing page: 1

[%] Indexed 1 Images on Page 1.


[%] Downloading Image #1 from https://cdn.wallpapersafari.com/54/57/VFmTa6.png
[%] File Downloaded !



[%] Done. Downloaded 1 images.

Please show your support here
https://www.buymeacoffee.com/gurugaurav


[MoviePy] >>>> Building video 2.mp4
[MoviePy] Writing video 2.mp4


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 

[MoviePy] >>>> Building video Final.mp4
[MoviePy] Writing audio in FinalTEMP_MPY_wvf_snd.mp3


100%|██████████| 763/763 [00:01<00:00, 709.01it/s]

[MoviePy] Done.
[MoviePy] Writing video Final.mp4



100%|██████████| 995/995 [00:53<00:00, 18.72it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Final.mp4 

